<a href="https://colab.research.google.com/github/kavyajeetbora/nlp_rag/blob/master/notebooks/05_LangChain_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Outline

Here we creating an LLM application that generates restaurant names and an entire menu based on the cuisine that user chooses.

This handson exercise is a getting started tutorial with Langchain

## Setup environment


In [22]:
!pip install -q langchain langchain_community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00


In [25]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv

Setup environment variables

In [26]:
from google.colab import files
uploaded = files.upload()
if uploaded:
    print("Uploaded")
    load_dotenv('.env')
    openai_api_key = os.getenv("OPENAI_API_KEY")

Saving .env to .env (2)
Uploaded


In [28]:
os.environ["OPEN_KEY_API"] = openai_api_key

## Setup the LLM model

In [29]:
llm = OpenAI(temperature=0.6)

In [30]:
name = llm("Suggest a  name for an Indian Restaurant")
print(name)



"Spice Nation"


## Creating a prompt template

In [71]:
prompt_template = PromptTemplate(
    name="Restaurant Names",
    input_variables= ['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
prompt_template.format(cuisine = "Mexican")

'I want to open a restaurant for Mexican food. Suggest a fancy name for this.'

## Create a chain

In [72]:
chain = LLMChain(llm=llm, prompt=prompt_template, output_key='restaurant_name')
chain.invoke("Mexican")

{'cuisine': 'Mexican', 'restaurant_name': '\n\n"La Cantina del Sol"'}

In [74]:
prompt_template_items = PromptTemplate(
    name="Menu Items",
    input_variables = ["restaurant_name"],
    template= "Suggest some menu items for the {restaurant_name} in comma separated values"
)

prompt_template_items.format(restaurant_name="Indian Bistro")

'Suggest some menu items for the Indian Bistro in comma separated values'

In [75]:
item_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key = "menu_items")
item_chain.invoke({'restaurant_name': 'Arabic Delights'})

{'restaurant_name': 'Arabic Delights',
 'menu_items': "\n\n1. Hummus, Falafel, and Baba Ghanoush Platter\n2. Shawarma Wrap with Chicken, Beef, or Lamb\n3. Tabouleh Salad with Fresh Herbs and Lemon Dressing\n4. Kebab Skewers with Grilled Vegetables and Tzatziki Sauce\n5. Fattoush Salad with Pita Croutons and Sumac Dressing\n6. Stuffed Grape Leaves with Rice and Herbs\n7. Lamb Kofta with Spiced Rice and Yogurt Sauce\n8. Chicken Shawarma Bowl with Rice, Vegetables, and Tahini Sauce\n9. Spinach and Feta Fatayer (Savory Pastries)\n10. Baklava for Dessert\n11. Lentil Soup with Lemon and Cumin\n12. Grilled Halloumi Cheese with Za'atar and Olive Oil\n13. Chicken or Beef Shawarma Plate with Rice, Salad, and Pita Bread\n14. Falafel Wrap with Tahini Sauce and Pickled Vegetables\n15. Lamb Mansaf with Yogurt Sauce and Rice."}

## Create sequential chain

In [77]:
from langchain.chains import SequentialChain

seq_chain = SequentialChain(
    chains = [chain, item_chain],
    input_variables = ['cuisine'],
    output_variables = ["restaurant_name", "menu_items"]
)

In [79]:
seq_chain.invoke("Pakistani")

{'cuisine': 'Pakistani',
 'restaurant_name': '\n\n"Mughal Spice Bistro"',
 'menu_items': '\n\n\n1. Chicken Tikka Masala, Naan Bread, Basmati Rice\n2. Lamb Rogan Josh, Garlic Naan, Vegetable Samosas\n3. Butter Chicken, Vegetable Biryani, Tandoori Cauliflower\n4. Palak Paneer, Garlic Naan, Aloo Gobi\n5. Tandoori Chicken, Dal Makhani, Mango Lassi\n6. Malai Kofta, Jeera Rice, Onion Bhaji\n7. Chicken Korma, Garlic Naan, Saag Aloo\n8. Lamb Vindaloo, Vegetable Pakoras, Mango Chutney\n9. Prawn Curry, Vegetable Biryani, Raita\n10. Paneer Tikka Masala, Garlic Naan, Vegetable Jalfrezi'}